In [1]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup

### Requests
- api 사용 : json파싱을 한다.
- 네이버 주식 페이지에서 주식 데이터를 가져와 데이터 프레임으로 만들기

In [2]:
def __json_url(cat=0,page_num=1):
    '''
    cat : 0 = KOSPI
        : 2 = KOSDAQ
    '''
    return 'https://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok={0}&pageSize=20&page={1}'.format(cat,page_num)

def scrapying_naver_stock_data(cat=0,page_num=1):
    url = __json_url(cat,page_num)
    response = requests.get(url)
    json_info = response.json()
    comp_info = json_info['result']['itemList']
    df = pd.DataFrame(columns=['종목','시세','전일비','등락율','시가총액','거래량'])
    for company in comp_info:
        df.loc[len(df)] = {
        "종목":company["nm"],
        "시세":company["nv"],
        "전일비":company["cv"],
        "등락율":company["cr"],
        "시가총액":company["mks"],
        "거래량":company["aq"],
        }
    return df

In [3]:
scrapying_naver_stock_data(0,1)

,종목,시세,전일비,등락율,시가총액,거래량
0,삼성전자,44900.0,-1200.0,-2.60,2882277.0,17726813.0
1,SK하이닉스,75900.0,-2900.0,-3.68,552554.0,8374338.0
2,셀트리온,279500.0,2000.0,0.72,350482.0,403404.0
3,삼성전자우,36100.0,-1100.0,-2.96,326210.0,1551518.0
4,삼성바이오로직스,458000.0,-1000.0,-0.22,303036.0,100229.0
5,현대차,133000.0,-1000.0,-0.75,284179.0,690740.0
6,POSCO,308000.0,-500.0,-0.16,268535.0,274091.0
7,LG화학,352000.0,1000.0,0.28,248485.0,170885.0
8,NAVER,732000.0,2000.0,0.27,241287.0,58603.0
9,삼성물산,125000.0,-1500.0,-1.19,237113.0,252019.0


### BeautifulSoup
- bs4 사용 :html element를 selector를 사용해 가져온다.
- 네이버 검색어 순위를 가져와 데이터 프레임으로 만들기

In [4]:
def give_me_naver_rank():
    response = requests.get('https://naver.com')
    dom = BeautifulSoup(response.content , 'html.parser')
    keywords = dom.select('.ah_roll .ah_l .ah_item')
    rank_ls = [keyword.select_one(".ah_r").text for keyword in keywords]
    content_ls = [keyword.select_one('.ah_k').text for keyword in keywords]
    df = pd.DataFrame()
    df['rank'] = rank_ls
    df['content'] = content_ls
    return df

In [5]:
give_me_naver_rank()

,rank,content
0,1,박환희
1,2,bap
2,3,ufc
3,4,안시성
4,5,선미
5,6,롯데시네마
6,7,로또823회당첨번호
7,8,진학어플라이
8,9,메르스
9,10,미스터 션샤인


In [6]:
response = requests.get('https://daum.net')
dom = BeautifulSoup(response.content , 'html.parser')
keywords = dom.select('#mArticle ol.list_hotissue.issue_row.list_mini > li')
rank_ls = [keyword.select_one(".ir_wa").text for keyword in keywords]
content_ls = [keyword.select_one('.link_issue').text for keyword in keywords]
df = pd.DataFrame()
df['rank'] = rank_ls
df['content'] = content_ls

In [7]:
df

,rank,content
0,1위,오사카 나오미
1,2위,송해
2,3위,bap
3,4위,선미
4,5위,박환희
5,6위,유현주
6,7위,판빙빙
7,8위,ufc228
8,9위,현지에서 먹힐까
9,10위,안시성


### CSS_selector
- `#` : ID
- `.` : class
- `>` : right below element